In [1]:
import re

def replace_currency_symbols(text):
    # Dictionary mapping currency symbols to their acronyms
    currency_dict = {
        '£': 'GBP ',   # British Pound
        '€': 'EUR ',   # Euro
        '$': 'USD ',   # US Dollar
        '¥': 'JPY ',   # Japanese Yen
        '₹': 'INR ',   # Indian Rupee
        '₽': 'RUB ',   # Russian Ruble
        '₩': 'KRW ',   # South Korean Won
        '฿': 'THB ',   # Thai Baht
        '₺': 'TRY ',   # Turkish Lira
        '₪': 'ILS '    # Israeli Shekel
    }

    # Regular expression pattern to match any of the currency symbols
    pattern = re.compile('|'.join(re.escape(symbol) for symbol in currency_dict.keys()))

    # Function to replace a matched symbol with its acronym
    def replace_symbol(match):
        return currency_dict[match.group(0)]

    # Replace all currency symbols in the text
    return pattern.sub(replace_symbol, text)
def clean_text(text):
    # Add a space before and after punctuation marks
    text = re.sub(r'([.,!?;:])', r' \1 ', replace_currency_symbols(text))
    
    # Remove newlines and tabs
    text = text.replace('\n', ' ').replace('\t', ' ')
    
    # Remove any extra spaces that might have been introduced
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text
# Example usage
text = "The price is £50, €30, and $20."
result = replace_currency_symbols(text)
print(result)  # Output: "The price is GBP50, EUR30, and USD20."


The price is GBP 50, EUR 30, and USD 20.


In [2]:
import pandas as pd

cnbc = pd.read_csv('data/raw_data/cnbc_headlines.csv')
guardian = pd.read_csv('data/raw_data/guardian_headlines.csv')
reuters = pd.read_csv('data/raw_data/reuters_headlines.csv') 


In [3]:
print(len(cnbc) + len(guardian) + len(reuters))

53650


In [4]:
cnbc['Time'] = pd.to_datetime(cnbc['Time'].replace('ET ',''), errors = 'coerce').dt.strftime('%Y-%m-%d')
guardian['Time'] = pd.to_datetime(guardian['Time'], errors = 'coerce').dt.strftime('%Y-%m-%d')
reuters['Time'] = pd.to_datetime(reuters['Time'], errors = 'coerce').dt.strftime('%Y-%m-%d')

C:\Users\Jon\AppData\Local\Temp\ipykernel_25972\326647827.py:1: FutureWarning: Parsed string " 7:51  PM ET Fri, 17 July 2020" included an un-recognized timezone "ET". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  cnbc['Time'] = pd.to_datetime(cnbc['Time'].replace('ET ',''), errors = 'coerce').dt.strftime('%Y-%m-%d')
C:\Users\Jon\AppData\Local\Temp\ipykernel_25972\326647827.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cnbc['Time'] = pd.to_datetime(cnbc['Time'].replace('ET ',''), errors = 'coerce').dt.strftime('%Y-%m-%d')
C:\Users\Jon\AppData\Local\Temp\ipykernel_25972\326647827.py:1: FutureWarning: Parsed string " 7:33  PM ET Fri, 17 July 2020" included an un-recognized timezone "ET". Dropping unreco

In [5]:
cnbc = cnbc.dropna()
guardian = guardian.dropna()
reuters = reuters.dropna()

In [6]:
print(len(cnbc) + len(guardian) + len(reuters))

53330


In [7]:
import numpy as np
all_dates = np.unique(np.concatenate((cnbc['Time'].unique(), guardian['Time'].unique(), reuters['Time'].unique())))

In [8]:
combined_news = pd.concat([cnbc[['Time','Headlines']], guardian[['Time','Headlines']]], axis=0)
combined_news = pd.concat([combined_news[['Time','Headlines']], reuters[['Time','Headlines']]], axis=0)

In [9]:
combined_news.sort_values('Time')
combined_news = combined_news.rename(columns={'Time':'date','Headlines':'headlines'})

In [10]:
for i in range(len(combined_news)):
    combined_news['headlines'].iloc[i] = clean_text(combined_news['headlines'].iloc[i])

C:\Users\Jon\AppData\Local\Temp\ipykernel_25972\1809975150.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  combined_news['headlines'].iloc[i] = clean_text(combined_news['headlines'].iloc[i])


In [11]:
combined_news.to_csv('data/raw_data/combined_headlines.csv', index=False)

In [12]:
import yfinance as yf
from datetime import date, timedelta

def get_stock_data(ticker = 'AAPL', start_date = (date.today() - timedelta(days=365)).strftime('%Y-%m-%d'), end_date = date.today().strftime('%Y-%m-%d')):
    # Load the stock data
    data = yf.download(ticker, start=start_date, end=end_date)
    return data

In [25]:
prediction_period = 1
start_date = pd.to_datetime(combined_news['date'].iloc[-1]).strftime('%Y-%m-%d')
end_date = (pd.to_datetime(combined_news['date'].iloc[0]) + timedelta(days=14)).strftime('%Y-%m-%d')
sp500 = get_stock_data('SPY',start_date=start_date, end_date=end_date)

[*********************100%%**********************]  1 of 1 completed


In [26]:
sp500.to_csv('data/raw_data/spy_data.csv')

In [27]:
spy = pd.read_csv('data/raw_data/spy_data.csv')[['Date','Close']]

In [28]:
spy.columns

Index(['Date', 'Close'], dtype='object')

In [29]:
import numpy as np
combined_news['spy_close'] = np.nan
combined_news['spy_future_close'] = np.nan

C:\Users\Jon\AppData\Local\Temp\ipykernel_25972\2140812467.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_news['spy_close'] = np.nan
C:\Users\Jon\AppData\Local\Temp\ipykernel_25972\2140812467.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_news['spy_future_close'] = np.nan


In [30]:
for i in range(prediction_period,len(spy)):
    combined_news['spy_close'][combined_news['date']==spy['Date'].iloc[i-prediction_period]] = spy['Close'].iloc[i-prediction_period]
    combined_news['spy_future_close'][combined_news['date']==spy['Date'].iloc[i-prediction_period]] = spy['Close'].iloc[i]

C:\Users\Jon\AppData\Local\Temp\ipykernel_25972\3325685753.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  combined_news['spy_close'][combined_news['date']==spy['Date'].iloc[i-prediction_period]] = spy['Close'].iloc[i-prediction_period]


In [31]:
combined_news['price_movement'] = combined_news['spy_future_close'] - combined_news['spy_close']

C:\Users\Jon\AppData\Local\Temp\ipykernel_25972\4077164831.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_news['price_movement'] = combined_news['spy_future_close'] - combined_news['spy_close']


In [32]:
combined_news['move_percent'] = 100 * combined_news['price_movement'] / combined_news['spy_close']

C:\Users\Jon\AppData\Local\Temp\ipykernel_25972\338855478.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_news['move_percent'] = 100 * combined_news['price_movement'] / combined_news['spy_close']


In [33]:
combined_news

,date,headlines,spy_close,spy_future_close,price_movement,move_percent
0,2020-07-17,Jim Cramer : A better way to invest in the Cov...,321.720001,324.320007,2.600006,0.808158
1,2020-07-17,Cramer's lightning round : I would own Teradyne,321.720001,324.320007,2.600006,0.808158
3,2020-07-17,"Cramer's week ahead : Big week for earnings , ...",321.720001,324.320007,2.600006,0.808158
4,2020-07-17,IQ Capital CEO Keith Bliss says tech and healt...,321.720001,324.320007,2.600006,0.808158
5,2020-07-16,Wall Street delivered the 'kind of pullback I'...,320.790009,321.720001,0.929993,0.289907
...,...,...,...,...,...,...
32765,2018-03-20,Malaysia says never hired British data firm at...,270.950012,270.429993,-0.520020,-0.191925
32766,2018-03-20,Prosecutors search Volkswagen headquarters in ...,270.950012,270.429993,-0.520020,-0.191925
32767,2018-03-20,McDonald's sets greenhouse gas reduction targets,270.950012,270.429993,-0.520020,-0.191925
32768,2018-03-20,Pratt & Whitney to deliver spare A320neo engin...,270.950012,270.429993,-0.520020,-0.191925


In [34]:
for i in range(len(combined_news)-1):
    if pd.isna(combined_news['spy_close'].iloc[i]):
        combined_news['headlines'].iloc[i+1] = combined_news['headlines'].iloc[i] + combined_news['headlines'].iloc[i+1]

In [35]:
combined_news = combined_news.dropna()

In [36]:
combined_news.to_csv('data/1day_news_with_spy.csv', index=False)